In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%autosave 0
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

import sys
sys.path.insert(0,'..')

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')


Autosave disabled


In [2]:
# LOAD DATA
#{seqname:[shapearray, sequence, structure]}

import rna_io
data = rna_io.get_all_data('../data/RNA16.react', '../data/RNA16.dbn')
data.pop("R009")
data.pop("23sRNA")
print "done"


data for 'tRNA-asp' is corrupted, ignoring...
done


In [3]:
# TRAIN ON N-1, predict on rest
import simushape_representatives as sr
keys= data.keys()
predictions = list(sr.crosspredict(data, keys,cutoff=0.0000000001))


In [4]:
import rna_tools as rt

# EVALUATE 
for key,prediction in zip(keys,predictions):
    proba = rt.probability(data[key][2],data[key][1])
    annproba = rt.probability(data[key][2],data[key][1], prediction)
    print "%0.9f\t%0.9f\t%0.9f\t%d\t%d" % (proba, annproba, proba/annproba,  proba < annproba , data[key][2] == rt.fold(data[key][1]))

../rna_tools.py:33: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if react==None:
../rna_tools.py:43: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if react == None:


0.228434512	0.614614446	0.371671237	1	1
0.000000803	0.000000000	735244054616.599121094	0	0
0.060386882	0.321173558	0.188019469	1	1
0.000000000	0.000000000	6.055775599	0	0
0.000022349	0.000044902	0.497733985	1	0
0.000000010	0.000000000	98.668969886	0	0
0.235968953	0.634886238	0.371671237	1	1
0.000035202	0.000000001	49319.497386231	0	0
0.000000050	0.000000000	5699.335571254	0	0
0.000000069	0.000000000	1969843914.578686714	0	0
0.096670345	0.123308179	0.783973498	1	0
0.000000381	0.000000000	3336.470342538	0	0
0.000000000	0.000000000	9735.565618440	0	0


In [ ]:
# todo: subopt, regressor